In [9]:
import numpy as np

class Gate():

    # Constructor to set up Gate class
    def __init__(self, name, matrix):
        self.name = name
        self.matrix = np.array(matrix, dtype=complex)
        if self.matrix.shape != (2,2):
            raise ValueError("Matrix has to be 2x2")

    def apply(self, state_vector):
        state_vector = np.array(state_vector, dtype=complex)

        if state_vector.shape != (2,2):
            raise ValueError("State vector needs to be a 2x1 vector")
            
        return self.matrix @ state_vector

    def toString(self):
        return f"{self.name} Gate: {self.matrix}"


'''
state - state vector
gate - gate being applied to state vector
q - index of the qubit
n - number of qubits 

===============================================Explanation=========================================
Instead of making a 2^n * 2^n state matrix, we will reshape the state matrix to a 2^n tensor vector. By doing this, each
index in the state vector corresponds to one of the computational basis states. A gate acts on only one qubit, but the full
state spans all n qubits. So we need to apply the gate matrix to the correct subspace while keeping other qubits unchanged.

To apply the gate:
1. move target axis to front, multiply, and then mvoe it back
2. Apply the gate along the axis using tensordot
3. move axes back to original order by undoing transpose
4. flatten the tensor

'''
def apply_single_qubit(state, gate, q, n):
    # Reshape to 2 x 2 x ... x 2 tensor
    psi = state.reshape((2,)*n)

    # Move target axis to front, multiply, move back
    axes = list(range(n))
    axes[0], axes[q] = axes[q], axes[0]
    psi = np.transpose(psi, axes)

    # Apply gate on the first axis
    psi = np.tensordot(gate.matrix, psi, axes=([1],[0]))

    # Undo transpose
    inv_axes = np.argsort(axes)
    psi = np.transpose(psi, inv_axes)

    return psi.reshape(-1)

H = Gate("Hadamard", (1/np.sqrt(2)) * np.array([[1, 1], [1,-1]]))
print(H.toString())

# Test - Applies Hadamard Gate to |0>
test_state = np.array([1,0], dtype=complex)
res = apply_single_qubit(test_state, H, q=0, n=1)
print(f"Result:\n{res}")

Hadamard Gate: [[ 0.70710678+0.j  0.70710678+0.j]
 [ 0.70710678+0.j -0.70710678+0.j]]
Result:
[0.70710678+0.j 0.70710678+0.j]


In [11]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
